In [1]:
from mlflow.entities import run
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient, client
from pprint import pprint
import joblib
import os


def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]

    model_name = mlflow_config["registered_model_name"]

    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_registry_uri(remote_server_uri)

    # Search for no. of runs already completed.
    runs = mlflow.search_runs(experiment_ids=1)
    pprint(runs)
    lowest = runs["metrics.mae"].sort_values()(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)

            # Change the model's stage.
            client.transition_model_version_stage(
                name=model_name, version=current_version, stage="Production"
            )
        else:
            # Get the other models to Staging/None.
            current_version = mv["version"]
            pprint(mv, indent=4)
            # Change the model's stage.
            client.transition_model_version_stage(
                name=model_name, version=current_version, stage="Staging"
            )


log_production_model("../params.yaml")

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'int'